In [1]:
from keras.models import Sequential, Model, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.metrics import confusion_matrix
from keras.applications.vgg16 import VGG16
from keras import applications
from keras.utils.data_utils import Sequence
from keras import optimizers

from glob import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
model = VGG16(weights='imagenet', include_top=False)
result = model.output
result = GlobalAveragePooling2D()(result) #池化層  對2維輸入進行全域性平均值池化過濾
result = Dense(512, activation='relu')(result)
predictions = Dense(2, activation='sigmoid')(result)
inception_transfer = Model(inputs=model.input, outputs=predictions)
inception_transfer.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [3]:
inception_transfer.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0   

In [4]:
train_data  = 'train/'
test_data   = 'test/'
val_data    = 'val/'

In [5]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1. / 255,      #縮放   
    shear_range=0.2,       #以弧度逆时针方向剪切角度
    zoom_range=0.2,        #随機缩放範圍
    horizontal_flip=True)  #隨機水平翻轉

test_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_data,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [6]:
history_pretrained = inception_transfer.fit_generator(
    train_generator,
    epochs=16, shuffle = True, 
    verbose = 1, validation_data = validation_generator)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/16
326/326 [==============================] - 230s 705ms/step - loss: 0.3625 - accuracy: 0.8190 - val_loss: 0.8219 - val_accuracy: 0.6250
Epoch 2/16
326/326 [==============================] - 144s 443ms/step - loss: 0.2012 - accuracy: 0.9120 - val_loss: 1.2417 - val_accuracy: 0.6250
Epoch 3/16
326/326 [==============================] - 144s 441ms/step - loss: 0.1629 - accuracy: 0.9360 - val_loss: 0.3519 - val_accuracy: 0.8750
Epoch 4/16
326/326 [==============================] - 145s 443ms/step - loss: 0.1390 - accuracy: 0.9486 - val_loss: 0.2946 - val_accuracy: 0.8125
Epoch 5/16
326/326 [==============================] - 137s 421ms/step - loss: 0.1217 - accuracy: 0.9515 - val_loss: 0.7109 - val_accuracy: 0.6250
Epoch 6/16
326/326 [==============================] - 137s 421ms/step - loss: 0.1121 - accuracy: 0.9595 - val_loss: 0.3393 - val_accuracy: 0.8125
Epoch 7/16
326/326 [=========

In [7]:
# Evaluate the model
scores = inception_transfer.evaluate_generator(test_generator)
print('acc =',scores[1]*100)
inception_transfer.save('model.h5')

acc = 91.66666865348816


In [8]:
from pathlib import Path
# Load model for testing
model = load_model('model.h5')

test_dir = Path('test')

normal_cases_dir = test_dir / 'NORMAL'
pneumonia_cases_dir = test_dir / 'PNEUMONIA'

normal_cases = normal_cases_dir.glob('*.jpeg')
pneumonia_cases = pneumonia_cases_dir.glob('*.jpeg')

test_data = []
test_labels = []

In [9]:
import cv2
import tensorflow as tf

for img in normal_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (224, 224))
    if img.shape[2] == 1:
        img = np.dstack([img, img, img])
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = tf.keras.utils.to_categorical(0, num_classes=2)
    test_data.append(img)
    test_labels.append(label)

for img in pneumonia_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (224, 224))
    if img.shape[2] == 1:
        img = np.dstack([img, img, img])
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = tf.keras.utils.to_categorical(1, num_classes=2)
    test_data.append(img)
    test_labels.append(label)

test_data = np.array(test_data)
test_labels = np.array(test_labels)

print("Total number of test examples: ", test_data.shape)
print("Total number of labels:", test_labels.shape)

Total number of test examples:  (624, 224, 224, 3)
Total number of labels: (624, 2)


In [10]:
# Get predictions
from sklearn import metrics
preds = model.predict(test_data, batch_size=16)
preds = np.argmax(preds, axis=-1)
orig_test_labels = np.argmax(test_labels, axis=-1)

print(metrics.classification_report(orig_test_labels, preds))
print(metrics.confusion_matrix(orig_test_labels, preds))

              precision    recall  f1-score   support

           0       0.98      0.78      0.87       234
           1       0.88      0.99      0.93       390

    accuracy                           0.91       624
   macro avg       0.93      0.89      0.90       624
weighted avg       0.92      0.91      0.91       624

[[183  51]
 [  3 387]]
